In [19]:
# ===== Config =====
BASE_DIR    = r"K:\Kmis_Public\Bryant Le\Scanned Documents"
OUTPUT_XLSX = r"K:\Kmis_Public\Bryant Le\Scanned Documents\Onward_PDF_Index.xlsx"

# ===== Imports =====
import os, re
from pathlib import Path
from datetime import datetime
import pandas as pd

# ===== Regex Rules =====
RX_ON_ID = re.compile(
    r'\b(?:ON-?\d{2}-\d{3,6}(?:-\d{1,4})?|ON-\d{6,9}|ON\d{6,9})\b',
    re.IGNORECASE
)
RX_SPLIT_FIRST_DASH = re.compile(r'^(.*?)\s-\s(.+)$')
RX_PREFIX_NO_LINE = re.compile(r'^(?:Claim|File|Ref(?:erence)?)\s*No\.?\s*(.+?)\s-\s(.+)$', re.IGNORECASE)

def extract_claims_and_doc(stem: str):
    s = stem.strip()
    m = RX_PREFIX_NO_LINE.match(s)
    if m:
        left, doc = m.group(1).strip(), m.group(2).strip()
    else:
        m2 = RX_SPLIT_FIRST_DASH.match(s)
        if m2:
            left, doc = m2.group(1).strip(), m2.group(2).strip()
        else:
            left, doc = s, s
    claims_raw = RX_ON_ID.findall(left)
    seen, claims = set(), []
    for c in claims_raw:
        k = c.upper()
        if k not in seen:
            seen.add(k)
            claims.append(c.strip())
    return claims, doc

# ===== Scan Folders =====
rows, all_pdf_paths = [], []
for root, dirs, files in os.walk(BASE_DIR):
    if not os.path.basename(root).lower().startswith("onward"):
        continue
    for fname in files:
        if not fname.lower().endswith(".pdf"):
            continue
        fpath = os.path.join(root, fname)
        all_pdf_paths.append(fpath)
        stem = Path(fname).stem
        date_modified = datetime.fromtimestamp(os.path.getmtime(fpath))
        claims, doc_name = extract_claims_and_doc(stem)
        if claims:
            for c in claims:
                rows.append({
                    "Claim Number": c,
                    "Doc Name": doc_name,
                    "Date Modified": date_modified,
                    "File Path": fpath
                })
        else:
            rows.append({
                "Claim Number": "",
                "Doc Name": doc_name,
                "Date Modified": date_modified,
                "File Path": fpath
            })

df = pd.DataFrame(rows).sort_values(["Date Modified", "Doc Name", "Claim Number"]).reset_index(drop=True)

def _xl_escape(s: str) -> str:
    return s.replace('"', '""')

df["Link"] = df.apply(
    lambda r: f'=HYPERLINK("{_xl_escape(r["File Path"])}", "{_xl_escape(r["Doc Name"])}")',
    axis=1
)

# ===== Write Excel =====
engine = None
for cand in ("xlsxwriter", "openpyxl"):
    try:
        __import__(cand); engine = cand; break
    except ImportError:
        pass
if engine is None:
    raise ImportError("install xlsxwriter or openpyxl")

with pd.ExcelWriter(OUTPUT_XLSX, engine=engine, datetime_format="yyyy-mm-dd hh:mm") as writer:
    df.to_excel(writer, index=False, sheet_name="Onward PDFs")
    if engine == "xlsxwriter":
        ws = writer.sheets["Onward PDFs"]
        ws.set_column("A:A", 22)
        ws.set_column("B:B", 60)
        ws.set_column("C:C", 20)
        ws.set_column("D:D", 85)
        ws.set_column("E:E", 55)
        hyperlink_fmt = writer.book.add_format({'font_color': 'blue', 'underline': 1})
        for i in range(len(df)):
            ws.write_formula(i+1, 4, df.iloc[i, df.columns.get_loc("Link")], hyperlink_fmt)

if engine == "openpyxl":
    from openpyxl import load_workbook
    from openpyxl.styles import Font
    wb = load_workbook(OUTPUT_XLSX)
    ws = wb["Onward PDFs"]
    ws.column_dimensions["A"].width = 22
    ws.column_dimensions["B"].width = 60
    ws.column_dimensions["C"].width = 20
    ws.column_dimensions["D"].width = 85
    ws.column_dimensions["E"].width = 55
    for cell in ws["E"][1:]:
        cell.style = "Hyperlink"
        cell.font = Font(color="0000FF", underline="single")
    wb.save(OUTPUT_XLSX)

print(f"[Summary] PDFs found: {len(set(map(os.path.normpath, all_pdf_paths)))}")
print(f"[Summary] Rows written: {len(df)} → {OUTPUT_XLSX}")

[Summary] PDFs found: 163
[Summary] Rows written: 166 → K:\Kmis_Public\Bryant Le\Scanned Documents\Onward_PDF_Index.xlsx
